<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-and-paths" data-toc-modified-id="Data-and-paths-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data and paths</a></span></li><li><span><a href="#Prepare-data" data-toc-modified-id="Prepare-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare data</a></span></li><li><span><a href="#Train-a-model" data-toc-modified-id="Train-a-model-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Train a model</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Detection-models" data-toc-modified-id="Detection-models-3.0.1"><span class="toc-item-num">3.0.1&nbsp;&nbsp;</span>Detection models</a></span></li></ul></li></ul></li><li><span><a href="#Test-model-on-an-image" data-toc-modified-id="Test-model-on-an-image-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Test model on an image</a></span></li><li><span><a href="#HEJ!" data-toc-modified-id="HEJ!-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>HEJ!</a></span></li></ul></div>

This notebook is for training YOLOv3 models as described in https://imageai.readthedocs.io/en/latest/customdetection/index.html

In [ ]:
# Show python version in output using sys
import sys
print(sys.version)

In [ ]:
import pandas as pd
import json
import os
from Json2PascalVoc.Converter import Converter

In [ ]:
# !pip install keras


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# we need to crop images
from PIL import Image


In [ ]:
from imageai.Detection.Custom import DetectionModelTrainer
from imageai.Detection.Custom import CustomObjectDetection


## Data and paths

In [ ]:
path_home = os.getcwd()

#data_path = os.path.join(path_home, "data")
data_path = path_home
print(data_path)
print(path_home)

In [ ]:
# Images
image_path = os.path.join(data_path, 'Annotation/AnnotationImagesSeabirds')
image_path_a1 = os.path.join(image_path, 'Adult birds')
image_path_a2 = os.path.join(image_path, 'Chicks')
image_path_a3 = os.path.join(image_path, 'Egg')


## Prepare data

Arrange data according to instructions at https://imageai.readthedocs.io/en/latest/custom/index.html

* To get started, you need prepare your dataset in the Pascal VOC Format and organize it as detailed below:
* Decide the type of object(s) you want to detect and collect about 200 (minimum recommendation) or more picture of each of the object(s)
* Once you have collected the images, you need to annotate the object(s) in the images. You can use a tool like LabelIMG to generate the annotations for your images.
* Once you have the annotations for all your images, create a folder for your dataset (E.g headsets) and in this parent folder, create child folders train and validation
* In the train folder, create images and annotations sub-folders. Put about 70-80% of your dataset of each object’s images in the images folder and put the corresponding annotations for these images in the annotations folder.
* In the validation folder, create images and annotations sub-folders. Put the rest of your dataset images in the images folder and put the corresponding annotations for these images in the annotations folder.

In [ ]:
# Create new folders (unless they exist) for trainng and test data in the data folder
# Skip this if done manually
# paths:
train_data_path = os.path.join(data_path, 'detection', 'train')
test_data_path = os.path.join(data_path, 'detection', 'validation')
train_im_data_path = os.path.join(train_data_path, 'images')
test_im_data_path = os.path.join(test_data_path, 'images')
train_annot_data_path = os.path.join(train_data_path, 'annotations')
test_annot_data_path = os.path.join(test_data_path, 'annotations')
    
try:
    # create training data folder
    #os.mkdir(os.path.join(data_path, 'detection'))
    #os.mkdir(train_data_path)
    #os.mkdir(test_data_path)
    
    os.mkdir(train_im_data_path)
    os.mkdir(test_im_data_path)
    os.mkdir(train_annot_data_path)
    os.mkdir(test_annot_data_path)
except Exception:
    print('Failed! Folders already exist?')

In [ ]:
# Read CSV file with JSON formatted information for each training image
path1 = os.path.join(data_path, 'Annotation', 'Export AIIOS Birds_v2.csv')
df = pd.read_csv(path1, sep=';', header=0)
print(df.shape)



In [ ]:
# Split into training data and test data
df_train = df.sample(frac=0.8, axis=0)
print(df_train.shape)
df_train.head(10)

In [ ]:
# Remaining used for testing
df_test = df.drop(df_train.index)
print(df_test.shape)
df_test.head()

In [ ]:
template_json = {
   "data":[
      {
         "annotation":{
            "folder":"class1",
            "filename":"_ADC0362.jpg",
            "path":"~/Desktop/Dev/data/foo/train/class1/_ADC0362.jpg",
            "source":{
               "database":"Unknown"
            },
            "size":{
               "width":1500,
               "height":1500,
               "depth":3
            },
            "segmented":0,
            "object":[
               {
                  "name":"class1",
                  "pose":"Unspecified",
                  "truncated":0,
                  "difficult":0,
                  "bndbox":{
                     "xmin":579,
                     "ymin":584,
                     "xmax":924,
                     "ymax":1120
                  }
               },
               {
                  "name":"class1",
                  "pose":"Unspecified",
                  "truncated":0,
                  "difficult":0,
                  "bndbox":{
                     "xmin":120,
                     "ymin":400,
                     "xmax":1150,
                     "ymax":800
                  }
               }

            ]
         }
      },
   ]
}

In [ ]:
def json2json(f_name, f_path, dict_in, image_size, folder=""):
    # 
    dict_out = template_json.copy()
    temp1 = dict_out['data'][0]['annotation']
    
    temp1["folder"] = folder
    temp1["filename"] = f_name
    temp1["path"] = f_path
    temp1["size"] = image_size
    
    temp1["object"] = []
    ok = False
    for ii, obj in enumerate(dict_in['objects']):
        if len(obj['boundaries']) > 0:
            object_boundaries = obj['boundaries'][0]['boundaryPoints']
            x_vals = [x['coords'][0] for x in object_boundaries]
            y_vals = [x['coords'][1] for x in object_boundaries]
            left = min(x_vals)
            right = max(x_vals)
            bottom = max(y_vals)
            top = min(y_vals)  
        
            tmp =  {
                      "name": obj['type'],
                      "pose":"Unspecified",
                      "truncated":0,
                      "difficult":0,
                      "bndbox":{
                         "xmin":left,
                         "ymin":top,
                         "xmax":right,
                         "ymax":bottom,
                      }
            }
            temp1["object"].append(tmp) 
            ok = True
        else:
            print('Missing coordintaes, skipping')
            print(obj['boundaries'])
    return(dict_out, ok)

In [ ]:
# test
fname = 'Farallon3_20190525_221309_No001.jpg'
f_path = image_path
dict_in = data
im_size = { "width":1500,
               "height":1500,
               "depth":3,
          }
json2json(fname, f_path, dict_in, im_size)

In [ ]:
data

In [ ]:
# Prepare training  data
image_nr = 0

xml_list = []
num_im = 0
for ind in df_train.index:
    p1_str = df_train.loc[ind].get_values()
    pname, json_string = p1_str[0].split(',', 1)
    print('File name: ', pname)
    # Get objects
    data = json.loads(json_string)
    #data = json.dumps(json_string)
    # obj_list = data['objects']
    
    if len(data['objects']) > 0: 
        if len(data['objects'][0]['boundaries']) > 0:
            
            #Read image
            success = True        
            if pname in os.listdir(image_path):
                im = Image.open(os.path.join(image_path, pname))
                f_path = image_path
            elif pname in os.listdir(image_path_a1):
                im = Image.open(os.path.join(image_path_a1, pname))
                f_path = image_path_a1
            elif pname in os.listdir(image_path_a2):
                f_path = image_path_a2
                im = Image.open(os.path.join(image_path_a2, pname))
            elif pname in os.listdir(image_path_a3):
                im = Image.open(os.path.join(image_path_a3, pname))
                f_path = image_path_a3
            else:
                success = False
                print('Image {} not found!'.format(pname))
                # crash_me_now()

            if success:
               
                im_size = {'width': im.size[0], 'height': im.size[1], 'depth': 3}
                folder = train_data_path # f_path.split('/')[-1]

                new_data, success = json2json(pname, f_path, data, im_size, folder)
                
                if success:
                    num_im += 1
                    fname = pname.split('.')[0]
                    path_out = os.path.join(train_data_path, fname + '.json')

                    with open(path_out, 'w') as f:
                        f.write(json.dumps(new_data))

                    #jsonfile = open(path_out, 'w')
                    #jsonfile.write(json.dumps(new_data))

                    # print(new_data)


                    myConverter = Converter()
                    # returns a Converter Object
                    myConverter.convertJsonToPascal(path_out)
                    # Converts Json to PascalVOC XML and saves the XML file to the related file path
                    xml_list.append(fname + '.xml')
        
        
print('Done')  
print('Number of images: ', num_im)
print('Last object:')
plt.imshow(im)
plt.show()


In [ ]:
# Prepare validation  data
image_nr = 0

xml_list = []
num_im = 0
for ind in df_test.index:
    p1_str = df_test.loc[ind].get_values()
    pname, json_string = p1_str[0].split(',', 1)
    print('File name: ', pname)
    # Get objects
    data = json.loads(json_string)
    #data = json.dumps(json_string)
    # obj_list = data['objects']
    
    if len(data['objects']) > 0: 
        if len(data['objects'][0]['boundaries']) > 0:
            
            #Read image
            success = True        
            if pname in os.listdir(image_path):
                im = Image.open(os.path.join(image_path, pname))
                f_path = image_path
            elif pname in os.listdir(image_path_a1):
                im = Image.open(os.path.join(image_path_a1, pname))
                f_path = image_path_a1
            elif pname in os.listdir(image_path_a2):
                f_path = image_path_a2
                im = Image.open(os.path.join(image_path_a2, pname))
            elif pname in os.listdir(image_path_a3):
                im = Image.open(os.path.join(image_path_a3, pname))
                f_path = image_path_a3
            else:
                success = False
                print('Image {} not found!'.format(pname))
                # crash_me_now()

            if success:
               
                im_size = {'width': im.size[0], 'height': im.size[1], 'depth': 3}
                folder = test_data_path # f_path.split('/')[-1]

                new_data, success = json2json(pname, f_path, data, im_size, folder)
                
                if success:
                    num_im += 1
                    fname = pname.split('.')[0]
                    path_out = os.path.join(test_data_path, fname + '.json')

                    with open(path_out, 'w') as f:
                        f.write(json.dumps(new_data))

                    #jsonfile = open(path_out, 'w')
                    #jsonfile.write(json.dumps(new_data))

                    # print(new_data)


                    myConverter = Converter()
                    # returns a Converter Object
                    myConverter.convertJsonToPascal(path_out)
                    # Converts Json to PascalVOC XML and saves the XML file to the related file path
                    xml_list.append(fname + '.xml')
        
        
print('Done')  
print('Number of images: ', num_im)
print('Last object:')
plt.imshow(im)
plt.show()


## Train a model

FIXME

#### Detection models

* Need to install keras, 'pip install keras' should work
* Input data stored in different format, maybe can be use as it is in Annotation folder? 
    * Probably not. See instructions.

In [ ]:
model_type_list = [ 'YOLOv3']
model_type = model_type_list[0]
print('Model type: ', model_type)
train_data_path = os.path.join(data_path, 'detection')

In [ ]:
model_trainer = DetectionModelTrainer()

if model_type == 'YOLOv3':
    model_trainer.setModelTypeAsYOLOv3()
else:
    print('Not supported')
    
model_trainer.setDataDirectory(train_data_path)

model_trainer.setTrainConfig(object_names_array=['AdultBird', 'Chick', 'Egg'], batch_size=1, num_experiments=1,
                            train_from_pretrained_model="pretrained-yolov3.h5")
model_trainer.trainModel()


In [ ]:
data_path

## Test model on an image

In [ ]:
# FIXME this is copied code
detector = CustomObjectDetection()
detector.setModelTypeAsYOLOv3()
detector.setModelPath("hololens-ex-60--loss-2.76.h5")
detector.setJsonPath("detection_config.json")
detector.loadModel()
detections = detector.detectObjectsFromImage(input_image="holo1.jpg", output_image_path="holo1-detected.jpg")
for detection in detections:
    print(detection["name"], " : ", detection["percentage_probability"], " : ", detection["box_points"])

## HEJ!

By continuing to browse the site, you are agreeing to our use of cookies.